# Day 15
https://adventofcode.com/2017/day/15

In [1]:
import aocd
data = aocd.get_data(year=2017, day=15)

In [2]:
from collections import deque
import regex as re

In [3]:
class Judge():
    
    def __init__(self, comparisons):
        self.comparisons = comparisons
        self.count = 0
        self.queues = {
            'A': deque(),
            'B': deque(),
        }
    
    @property
    def completed(self):
        return self.comparisons <= 0
    
    def check(self):
        while len(self.queues['A']) > 0 and len(self.queues['B']) > 0:
            a = self.queues['A'].popleft()
            b = self.queues['B'].popleft()
            self.comparisons -= 1
            if a == b:
                self.count += 1
    
    def report(self, source, number):
        if not self.completed:
            self.queues.get(source, deque()).append(number)
            self.check()

In [4]:
class Generator():
    
    def __init__(self, name, start_value, judge, filtered_judge):
        self.name = name
        self.value = start_value
        self.factor = 16807 if name == 'A' else 48271
        self.check = 4 if name == 'A' else 8
        self.judge = judge
        self.filtered_judge = filtered_judge
    
    @staticmethod
    def hexstart(number):
        return f'{number:{0}16b}'[-16:]
    
    def run(self):
        hex_value = self.hexstart(self.value)
        
        self.judge.report(self.name, hex_value)
        if self.value % self.check == 0:
            self.filtered_judge.report(self.name, hex_value)
        
        self.value = (self.value * self.factor) % 2147483647

In [5]:
re_generators = re.compile(r'Generator (\w) starts with (\d+)')
def read_start_values(text):
    return dict((name, int(val)) for name, val in re_generators.findall(text))

In [6]:
def run_full_check(start_values):
    judges = (
        Judge(40_000_000),
        Judge(5_000_000),
    )
    generators = (
        Generator('A', start_values['A'], *judges),
        Generator('B', start_values['B'], *judges),
    )
    
    while not all(judge.completed for judge in judges):
        for gen in generators:
            gen.run()
    
    return tuple(judge.count for judge in judges)

In [7]:
values = read_start_values(data)
p1, p2 = run_full_check(values)
print(f'Part 1: {p1}')
print(f'Part 2: {p2}')

Part 1: 609
Part 2: 253
